In [1]:
import pandas as pd
import json

In [8]:
def set_or_value(x):
    if len(set(x)) > 1:
        return list(set(x))
    else:
        return x.iloc[0]

camera_link_format = '<a href=\"{0}\" target=\"_blank\"> <img src=\"{0}\" width=100%> </a>' # HTML format string for an image that links to its source

def make_camera_image_list(cameras):
    if type(cameras) is str:
        return camera_link_format.format(cameras)
    else:
        return '<br>'.join([camera_link_format.format(x) for x in cameras])

In [20]:
traffic_cameras = pd.read_json("https://511.alberta.ca/api/v2/get/cameras")
traffic_cameras.columns = [s.lower() for s in traffic_cameras.columns]
traffic_cameras['id_prefix'] = [s.split('.')[0] for s in traffic_cameras['id']]
traffic_cameras = traffic_cameras.groupby('id_prefix').aggregate(set_or_value).reset_index() # Aggregate any entries of the same camera ID prefix for multiple rows (usually 3 camera setups at interchanges). These entries consistently share the same lat/long.
traffic_cameras = traffic_cameras.rename(index=str, columns={"id_prefix": "id", "id": "sub_id"})
camera_features = []

for index, row in traffic_cameras.iterrows():
    row['popup'] = '<h2>' + row['name'] + '</h2>' + \
            make_camera_image_list(row['url']) + '<br>' + \
            str(row['description']) # TODO better handling of sets of differing descriptions
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']]
        },
        'properties': row.to_dict()
    }
    camera_features.append(feature)

camera_dict = {
    'type': 'FeatureCollection',
    'features': camera_features
}

print(json.dumps(camera_dict))

{"type": "FeatureCollection", "features": [{"geometry": {"type": "Point", "coordinates": [-115.473369, 55.316369]}, "properties": {"name": "Hwy2: West of Range Road 103 near Kinuso", "status": "Enabled", "organization": "AB", "sub_id": ["AB--ESS_AB_002-50 KINUSO.C1", "AB--ESS_AB_002-50 KINUSO.C2", "AB--ESS_AB_002-50 KINUSO.C3"], "popup": "<h2>Hwy2: West of Range Road 103 near Kinuso</h2><a href=\"https://511.alberta.ca/map/Cctv/ESS_AB_002-50 KINUSO.C3--1\" target=\"_blank\"> <img src=\"https://511.alberta.ca/map/Cctv/ESS_AB_002-50 KINUSO.C3--1\" width=100%> </a><br><a href=\"https://511.alberta.ca/map/Cctv/ESS_AB_002-50 KINUSO.C2--1\" target=\"_blank\"> <img src=\"https://511.alberta.ca/map/Cctv/ESS_AB_002-50 KINUSO.C2--1\" width=100%> </a><br><a href=\"https://511.alberta.ca/map/Cctv/ESS_AB_002-50 KINUSO.C1--1\" target=\"_blank\"> <img src=\"https://511.alberta.ca/map/Cctv/ESS_AB_002-50 KINUSO.C1--1\" width=100%> </a><br>WB, North side of HWY", "id": "AB--ESS_AB_002-50 KINUSO", "url":